# Select by Length

In [3]:
from langchain_core.prompts import FewShotPromptTemplate, PromptTemplate
from langchain_core.example_selectors.length_based import LengthBasedExampleSelector

examples = [
    {"food": "Kimchi", "category": "Korean food"},
    {"food": "Chocolate", "category": "dessert"},
    {"food": "Pasta", "category": "Italian food"},
    {"food": "Americano", "category": "Coffee"},
]

example_prompt = PromptTemplate(
    template="Food:{food} Category:{category}", input_variables=["food", "category"]
)
example_selector = LengthBasedExampleSelector(
    examples=examples, example_prompt=example_prompt, max_length=30
)
dynamic_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="What is the category of this food?",
    suffix="Food: {food}",
    input_variables=["food"],
)

output = dynamic_prompt.format(food="Korean BBQ")
print(len(output.split()), output)

20 What is the category of this food?

Food:Kimchi Category:Korean food

Food:Chocolate Category:dessert

Food:Pasta Category:Italian food

Food:Americano Category:Coffee

Food: Korean BBQ


In [4]:
output = dynamic_prompt.format(
    food="""Korean BBQ: Grilled meat, vibrant flavors, charcoal aroma. 
Tradition meets friends and family in a delightful dining experience, creating lasting memories.
"""
)
print(len(output.split()), output)

34 What is the category of this food?

Food:Kimchi Category:Korean food

Food:Chocolate Category:dessert

Food: Korean BBQ: Grilled meat, vibrant flavors, charcoal aroma. 
Tradition meets friends and family in a delightful dining experience, creating lasting memories.



In [5]:
food = (
    "Korean BBQ is a traditional cuisine where fresh meat "
    "is grilled with seasonings. The aroma of burning charcoal"
    "and rich flavors from special sauces create a delightful "
    "dining experience, often shared with family or friends."
)
output = dynamic_prompt.format(food=food)
print(len(output.split()), output)

42 What is the category of this food?

Food: Korean BBQ is a traditional cuisine where fresh meat is grilled with seasonings. The aroma of burning charcoaland rich flavors from special sauces create a delightful dining experience, often shared with family or friends.


In [7]:
%pip install nltk
# N-gram Selector

  Using cached click-8.1.7-py3-none-any.whl.metadata (3.0 kB)
  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 49.9 MB/s eta 0:00:00
Using cached click-8.1.7-py3-none-any.whl (97 kB)
Using cached joblib-1.4.2-py3-none-any.whl (301 kB)
Note: you may need to restart the kernel to use updated packages.


# N-gram Selector

In [8]:
from langchain_core.prompts import FewShotPromptTemplate, PromptTemplate
from langchain_community.example_selectors.ngram_overlap import NGramOverlapExampleSelector

examples = [
    {"food": "Kimchi is my favorite food.", "category": "Korean food"},
    {"food": "Chocolate is easy to gain weight food.", "category": "dessert"},
    {
        "food": "I love pasta; it's my favorite comfort food after work.",
        "category": "Italian food",
    },
    {
        "food": "Sipping an Americano, contemplating weight, a mindful morning routine begins.",
        "category": "Coffee",
    },
]

example_prompt = PromptTemplate(
    template="Food:{food} Category:{category}", input_variables=["food", "category"]
)
example_selector = NGramOverlapExampleSelector(
    examples=examples, example_prompt=example_prompt, threshold=-1.0
)
dynamic_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="What is the category in the food? .",
    suffix="Food: {food}",
    input_variables=["food"],
)

output = dynamic_prompt.format(food="Sushi is my favorite choice for party food")
print(output)

What is the category in the food? .

Food:Kimchi is my favorite food. Category:Korean food

Food:I love pasta; it's my favorite comfort food after work. Category:Italian food

Food:Chocolate is easy to gain weight food. Category:dessert

Food:Sipping an Americano, contemplating weight, a mindful morning routine begins. Category:Coffee

Food: Sushi is my favorite choice for party food


In [9]:
example_selector = NGramOverlapExampleSelector(
    examples=examples, example_prompt=example_prompt, threshold=0.0
)
dynamic_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="What is the category in the food? .",
    suffix="Food: {food}",
    input_variables=["food"],
)

output = dynamic_prompt.format(food="Sushi is my favorite choice for party food")
print(output)

What is the category in the food? .

Food:Kimchi is my favorite food. Category:Korean food

Food:I love pasta; it's my favorite comfort food after work. Category:Italian food

Food: Sushi is my favorite choice for party food


In [10]:
example_selector = NGramOverlapExampleSelector(
    examples=examples, example_prompt=example_prompt, threshold=-0.2
)
dynamic_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="What is the category in the food? .",
    suffix="Food: {food}",
    input_variables=["food"],
)

output = dynamic_prompt.format(food="Sushi is my favorite choice for party food")
print(output)

What is the category in the food? .

Food:Kimchi is my favorite food. Category:Korean food

Food:I love pasta; it's my favorite comfort food after work. Category:Italian food

Food:Chocolate is easy to gain weight food. Category:dessert

Food:Sipping an Americano, contemplating weight, a mindful morning routine begins. Category:Coffee

Food: Sushi is my favorite choice for party food


# Similarity based

In [13]:
%pip install langchain-chroma==0.1.4

Note: you may need to restart the kernel to use updated packages.


In [14]:
from langchain_openai import OpenAIEmbeddings
from langchain_core.prompts import FewShotPromptTemplate, PromptTemplate
from langchain_core.example_selectors.semantic_similarity import SemanticSimilarityExampleSelector
from langchain_chroma import Chroma

import os

os.environ["OPENAI_API_KEY"] = "{YOUR_OPENAI_KEY}"

examples = [
    {"input": "Happy.", "category": "emotion"},
    {"input": "BBQ", "category": "food"},
    {"input": "Golf", "category": "Sports"},
    {"input": "Student", "category": "Person"},
]

example_prompt = PromptTemplate(
    template="Input:{input} Category:{category}", input_variables=["input", "category"]
)
example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples,
    OpenAIEmbeddings(),
    Chroma,
    k=1,
)
dynamic_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="What is the category of the input? .",
    suffix="input: {input}",
    input_variables=["input"],
)

output = dynamic_prompt.format(input="Sushi")
print(output)

What is the category of the input? .

Input:BBQ Category:food

input: Sushi


## MMR Selector

In [16]:
from langchain_openai import OpenAIEmbeddings
from langchain_core.prompts import FewShotPromptTemplate, PromptTemplate
from langchain_core.example_selectors.semantic_similarity import (
    MaxMarginalRelevanceExampleSelector,
)
from langchain_chroma import Chroma

import os

os.environ["OPENAI_API_KEY"] = "{YOUR_OPENAI_KEY}"

examples = [
    {
        "input": "Please summarize the weather news.\n",
        "summary": "Today's weather: Sunny skies, mild temperatures,"
        " and a gentle breeze. Enjoy the pleasant conditions throughout the day!",
    },
    {
        "input": "Please summarize the economy news.\n",
        "summary": "Global stocks rise on positive economic data;"
        "inflation concerns persist. Tech sector outperforms; central banks closely monitor.",
    },
    {
        "input": "Please summarize retail news.\n",
        "summary": "Major retailer announces record-breaking sales during holiday shopping season",
    },
    {
        "input": "What is stock market trend?\n",
        "summary": "Investor optimism grows amid easing global trade tensions",
    },
    {
        "input": "Typhoon related news.\n",
        "summary": "IAirports and schools close ahead of approaching typhoon threat",
    },
]

example_prompt = PromptTemplate(
    template="Input:{input} Summary:{summary}", input_variables=["input", "summary"]
)
example_selector = MaxMarginalRelevanceExampleSelector.from_examples(
    examples,
    OpenAIEmbeddings(),
    Chroma,
    k=2,
)
dynamic_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    suffix="input: {input}\nSummary:",
    prefix="",
    input_variables=["input"],
)

output = dynamic_prompt.format(
    input="I want to know the economy trends and weather this week."
)
print(output)

Number of requested results 20 is greater than number of elements in index 9, updating n_results = 9


Input:Please summarize the weather news.
 Summary:Today's weather: Sunny skies, mild temperatures, and a gentle breeze. Enjoy the pleasant conditions throughout the day!

Input:What is stock market trend?
 Summary:Investor optimism grows amid easing global trade tensions

input: I want to know the economy trends and weather this week.
Summary:
